In [111]:
import pandas as pd
from random import randint
from random import sample
from random import uniform
import datetime
import numpy as np

In [112]:
# Считываем сущность product_category
categories = pd.read_excel('Data.xlsx', sheet_name='sr.1', index_col='id')
categories

,product_category
id,
1,Телефон
2,Монитор
3,Мышка
4,Клавиатура
5,Ноутбук
6,Игровая приставка
7,Наушники


In [113]:
# Считываем сущность products
products = pd.read_excel('Data.xlsx', sheet_name='sr.2', index_col='id')
print(products.shape)
products.head()

(157, 3)


,name,product_category_id,country_from
id,,,
1,Realme 9 Pro 8/128 Gb,1,Китай
2,Redmi Note 10 Pro 8/128 Gb,1,Китай
3,Poco X3 Pro 8/256 Gb,1,Китай
4,Redmi Note 11 Pro 5G 8/128 GB,1,Китай
5,Honor 50 8/128 Gb,1,Китай


In [114]:
# Считываем сущность sells - она пустая
sells = pd.read_excel('Data.xlsx', sheet_name='sr.3', index_col='id')
sells

,product_id,quantity,sell_date
id,,,


In [115]:
# Считываем сущность suppliers 
suppliers = pd.read_excel('Data.xlsx', sheet_name='sr.4', index_col='id')
suppliers

,supplier_name,address
id,,
1,Поставщик_1,Индия
2,Поставщик_2,США
3,Поставщик_3,Корея
4,Поставщик_4,Тайвань
5,Поставщик_5,Нидерланды
6,Поставщик_6,Швейцария
7,Поставщик_7,Япония
8,Поставщик_8,Англия
9,Поставщик_9,Турция


In [116]:
# Считываем сущность sup_to_prod - она пустая
sup_to_prod = pd.read_excel('Data.xlsx', sheet_name='sr.5', index_col='id')
sup_to_prod

,supplier_id,product_id
id,,


In [117]:
# Заполняем sup_to_prod
# Пусть каждый Поставщик имеет в своем каталоге от 30 до 45 наименований товаров
for sup_id in suppliers['supplier_name'].index:
    sup_products = sample(products.index.to_list(), randint(30, 45))
    for prod_id in sup_products:
        sup_to_prod.loc[len(sup_to_prod.index)] = [sup_id, prod_id]

In [118]:
sup_to_prod.loc[42:47]

,supplier_id,product_id
id,,
42,2,11
43,2,144
44,2,74
45,2,122
46,2,130
47,2,6


In [119]:
important_dates_list_first = ['2019-01-01', '2019-02-23', '2019-03-08', '2019-05-09', '2019-06-12']
important_dates_list_first = list(map(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'), important_dates_list_first))

important_dates_list = []
for i in range(4):
    for day in important_dates_list_first:
        important_dates_list.append(day + datetime.timedelta(days=365*i))

important_dates = pd.DataFrame(important_dates_list, columns=['Date'])[:16]
important_dates['StartDate'] = important_dates['Date'].apply(lambda x: x - datetime.timedelta(days=7))
important_dates['FinishDate'] = important_dates['Date'].apply(lambda x: x + datetime.timedelta(days=7))

start_day = important_dates.loc[0, 'StartDate']
finish_day = important_dates.loc[important_dates.index[-1], 'FinishDate']

all_time = (finish_day - start_day).days
all_days = [start_day + datetime.timedelta(days=day_num) for day_num in range(all_time+1)]

print(all_time)
important_dates

1109


,Date,StartDate,FinishDate
0,2019-01-01,2018-12-25,2019-01-08
1,2019-02-23,2019-02-16,2019-03-02
2,2019-03-08,2019-03-01,2019-03-15
3,2019-05-09,2019-05-02,2019-05-16
4,2019-06-12,2019-06-05,2019-06-19
5,2020-01-01,2019-12-25,2020-01-08
6,2020-02-23,2020-02-16,2020-03-01
7,2020-03-07,2020-02-29,2020-03-14
8,2020-05-08,2020-05-01,2020-05-15
9,2020-06-11,2020-06-04,2020-06-18


In [120]:
quality =pd.DataFrame(
    {
        'product_id': products.index,
        'quality': [randint(10000, 200000) for i in products.index]
    }
)

quality['mean_day_quality'] = quality['quality'].apply(lambda x: x // all_time)
quality['power'] = [uniform(1.5, 3) for i in products.index]

quality.head()


,product_id,quality,mean_day_quality,power
0,1,189386,170,2.407042
1,2,133366,120,1.723551
2,3,139988,126,2.109578
3,4,54693,49,2.338110
4,5,60760,54,2.596968


In [121]:
# заполняем sells
sels_product_id = quality.product_id.to_list() * len(all_days)
sels_sell_date = all_days * len(quality.product_id)

sells['product_id'] = sels_product_id
sells['sell_date'] = sels_sell_date

sells.head()

,product_id,quantity,sell_date
id,,,
0,1,NaN,2018-12-25
1,2,NaN,2018-12-26
2,3,NaN,2018-12-27
3,4,NaN,2018-12-28
4,5,NaN,2018-12-29


In [122]:
# заполняем знаковые даты

for prod_id in quality.product_id:
    for imp_day in important_dates.index:
        sdt = important_dates.loc[imp_day].StartDate
        fdt = important_dates.loc[imp_day].FinishDate
        imp_prd = sells.loc[(sells.sell_date >= sdt) & (sells.sell_date <= fdt) & (sells.product_id == prod_id)].index

        size = len(imp_prd)
        norm_value = quality.loc[quality.product_id == prod_id, 'mean_day_quality']
        mu = norm_value * quality.loc[quality.product_id == prod_id, 'power']
        sigma = mu - norm_value
        sells.loc[imp_prd, 'quantity'] = [int(abs(value)) for value in np.random.normal(loc=mu, scale=sigma, size=size)]

In [123]:
# заполняем остальные данные

for prod_id in quality.product_id:
    smp_day = sells.loc[(sells.quantity.isna()) & (sells.product_id==prod_id)].index
    size = len(smp_day)
    mu = quality.loc[quality.product_id == prod_id, 'mean_day_quality']
    sigma = mu * 0.2
    sells.loc[smp_day, 'quantity'] = [int(abs(value)) for value in np.random.normal(loc=mu, scale=sigma, size=size)]

In [124]:
sells.head()

,product_id,quantity,sell_date
id,,,
0,1,661,2018-12-25
1,2,191,2018-12-26
2,3,271,2018-12-27
3,4,154,2018-12-28
4,5,133,2018-12-29


In [125]:
writer = pd.ExcelWriter('DataFull.xlsx', engine='xlsxwriter')

categories.to_excel(writer, sheet_name='sr.1')
products.to_excel(writer, sheet_name='sr.2')
sells.to_excel(writer, sheet_name='sr.3')
suppliers.to_excel(writer, sheet_name='sr.4')
sup_to_prod.to_excel(writer, sheet_name='sr.5')

writer.save()

/var/folders/7g/1dqbz5hx5994z2r91kw1_k2m0000gn/T/ipykernel_26988/3136355800.py:9: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [126]:
sells
# нужно для каждого товара задать общее количество продаж
# | - задаем рандомно от 10 000 - 200 000 (+)
# | - нужно найти знаковые даты, когда обычно бывают скидки
# | - в знаковые даты сделать всплеск продаж - нормальное распрделение
# | - делать уменьшение цен в знаковые даты на 20% процентов

,product_id,quantity,sell_date
id,,,
0,1,661,2018-12-25
1,2,191,2018-12-26
2,3,271,2018-12-27
3,4,154,2018-12-28
4,5,133,2018-12-29
...,...,...,...
174265,153,24,2022-01-03
174266,154,158,2022-01-04
174267,155,69,2022-01-05
